## Economic Wellbeing Prediction - Regression Techniques

Predicting the measure of wealth across various regions in Africa


## Libraries used


In [ ]:
# libraries----
library(dplyr)
library(caret)
library(ggplot2)
library(reshape2)
library(glmnet)
library(MASS)
library(leaps)



## Dataset


In [ ]:
econ_wb <- read.csv("data/Train.csv")



## Conducting hypothesis tests and observing the correlation between different predictor variables


An analysis of variance test on the variable country. 
Looking at its statistical significance


In [ ]:
# ANOVA test  - country
oneway.test(Target~country, data = econ_wb)

ftest_country <- aov(Target~country, data = econ_wb)
summary(ftest_country)


High correlation. High test(f) statistic.
p value is less than the significance level. Null hypothesis is rejected. 


Correlation of the predictor variables


In [ ]:
# correlation - rest
cor_econ <- econ_wb[, c(5:19)]
cormat_econ <- round(cor(cor_econ),3)
cormat_econ_df <- as.data.frame(cormat_econ)
cormat_econ_df


Plot



In [ ]:
cor_t <- cormat_econ_df[, c(1,15)]


melt_cormat_econ <- melt(cormat_econ)

ggplot(melt_cormat_econ, aes(x=Var1, y=Var2, fill=value))+
  geom_tile()


Changing data types to factor. Changing binary level variable to numbers 1 and 0.



In [ ]:
econ_wb$country <- as.factor(econ_wb$country)
econ_wb$urban_or_rural <- ifelse(econ_wb$urban_or_rural == "U", 1, 0)
econ_wb$urban_or_rural <- as.factor(econ_wb$urban_or_rural)
econ_wb$year <- as.factor(econ_wb$year)


Drop ID column



In [ ]:
ec_data <- econ_wb[,-1]
colnames(ec_data)


Train-test split



In [ ]:
random <- sample(1:nrow(ec_data))
ec_data <- ec_data[random,]

split <- c(1: (nrow(ec_data) * 0.7)) 
train <- ec_data[split,]
test <- ec_data[-split,]


## Data pre-processing

Dummy variables


In [ ]:
dmy <- dummyVars("~.", data = train)
train_dmy <- data.frame(predict(dmy, newdata = train))

colnames(train_dmy)


Dropping unneeded columns



In [ ]:
train_dmy <- train_dmy[, -c(19:21,24,25,29)]
train_dmy <- train_dmy[, -c(1:18)]
colnames(train_dmy)


Same thing- but for the test data



In [ ]:
dmy <- dummyVars("~.", data = test)
test_dmy <- data.frame(predict(dmy, newdata = test))


test_dmy <- test_dmy[, -c(19:21,24,25,29)]
test_dmy <- test_dmy[, -c(1:18)]

colnames(test_dmy)


## Regression models

## Polynomial

First fit:


In [ ]:
pm1 <- lm(Target ~ poly(landcover_urban_fraction, 3) + poly(nighttime_lights, 3) + poly(ghsl_pop_density, 3) + poly(ghsl_built_1975_to_1990, 3) +  poly(ghsl_built_pre_1975, 3) +  poly(ghsl_built_2000_to_2014, 3) +  poly(ghsl_built_1990_to_2000, 3) +  poly(ghsl_not_built_up, 3) + poly(landcover_crops_fraction, 3) + poly(dist_to_shoreline, 3) + year.1998 + year.1999 + year.2005 + year.2006 + year.2007 + year.2010 + year.2011 + year.2012 + year.2013 + year.2014 + year.2015 + year.2016 + urban_or_rural.0 + urban_or_rural.1 , data = train_dmy)
summary(pm1)


Second fit- after altering some of the coefficients that were not statistically significant in the first



In [ ]:
pm2 <- lm(Target ~ poly(landcover_urban_fraction, 3) + poly(nighttime_lights, 3) + poly(ghsl_pop_density, 3) + poly(ghsl_built_1975_to_1990, 2) +  poly(ghsl_built_pre_1975, 1) + poly(ghsl_built_2000_to_2014, 3) +  poly(ghsl_not_built_up, 2) + poly(dist_to_shoreline, 1) +  poly(ghsl_built_1990_to_2000, 1) + year.1998 + year.1999 + year.2005 + year.2006 + year.2007 + year.2010 + year.2011 + year.2012 + year.2013 + year.2014 + year.2015 + year.2016 + urban_or_rural.0 + urban_or_rural.1, data = train_dmy)
summary(pm2)


Not much of a difference is observed


## Linear


In [ ]:
lm1 <- lm(Target ~ . , data = train_dmy)
summary(lm1)


The polynomial model is a better fit compared to the linear as per the adjusted r-squared values


## Ridge 


Looking at the optimal lambda value


In [ ]:
ridge_cv <- cv.glmnet(as.matrix(train_dmy[,-30]), train$Target, alpha = 0)
ridge_cv$lambda.min


Fitting the model at the optimal lambda value - observing the coefficients



In [ ]:
rm2 <- glmnet(as.matrix(train_dmy[,-30]), train_dmy$Target, alpha = 0, lambda = ridge_cv$lambda.min)
coef(rm2)


Predictions on the test dataset



In [ ]:
pred <- predict(rm2, s=ridge_cv$lambda.min, newx=as.matrix(test_dmy[,-30]))



Calculating R2 and RMSE for the ridge model



In [ ]:
R2(pred, test_dmy$Target)
RMSE(pred, test_dmy$Target)


## Lasso

Optimal lambda value:


In [ ]:
lasso_cv <- cv.glmnet(as.matrix(train_dmy[,-30]), train$Target, alpha = 1)
lasso_cv$lambda.min


Fitting the model at the optimal lambda value - observing the coefficients



In [ ]:
lsm1 <-  glmnet(as.matrix(train_dmy[,-30]), train_dmy$Target, alpha = 1, lambda = lasso_cv$lambda.min)
coef(lsm1)


Predictions on the test data



In [ ]:
pred2 <- predict(lsm1, s=lasso_cv$lambda.min, newx=as.matrix(test_dmy[,-30]))



Calculating R2 and RMSE



In [ ]:
R2(pred2, test_dmy$Target)
RMSE(pred2, test_dmy$Target)


## Stepwise

10 fold cross validation to estimate the average prediction error (RMSE)


In [ ]:
train_control <- trainControl(method = "cv", number = 10)



Fitting the model via backward elimination - removing the least significant values in each step
Specifying the tuning parameter nvmax, which corresponds to the maximum number of predictors to be incorporated in the model(5)
Model automatically chooses the best RMSE value that minimizes the error (lowest)


In [ ]:
stm1 <- train(
       Target~., data = train_dmy,
       method = "leapBackward", 
       tuneGrid = data.frame(nvmax = 1:5),
       trControl = train_control
)


Results:



In [ ]:
stm1$results
stm1$bestTune
summary(stm1$finalModel)
coef(stm1$finalModel, 5)


## Elastic Net

The caret package automatically selects the best alpha and lambda valuesa as tuning parameters for the model

Testing 10 different combinations of alpha and lambda


In [ ]:
elm2 <- train(
  Target ~., data = train_dmy, method = "glmnet",
  trControl = trainControl("cv", number = 10),
  tuneLength = 10
)


Best tune:



In [ ]:
elm2$bestTune



Coefficients of the best tune:



In [ ]:
coef(elm2$finalModel, elm2$bestTune$lambda)



Predictions on the test data



In [ ]:
pred <- elm2 %>% 
  predict(as.matrix(test_dmy[,-30]))


Calculating R2 and RMSE



In [ ]:
R2(pred, test_dmy$Target)
RMSE(pred, test_dmy$Target)


The R2 and RMSE values for all the regression models fit were almost the same value, with the polynomial model showing the best fit for the data, as per the test data


The various models were tested on the validation set, and the polynomial model sit had the best fit, with an RMSE of about 0.1203
